Install Dependencies

In [ ]:
!pip install datasets
!pip install evaluate
!pip install torch
!pip install accelerate -U
!pip install transformers[torch]
!pip install sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.1/542.1 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 24.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 21.6 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.31.0, but you have requests 2.32.3 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_6

In [ ]:
import evaluate
import torch
import numpy as np

from datasets import load_dataset, DatasetDict, Dataset

from transformers import (
    AutoTokenizer,
    AutoConfig,
    AutoModelForSeq2SeqLM,
    DataCollatorForSeq2Seq,
    TrainingArguments,
    Trainer
)

In [ ]:
# Allow file upload in google.colab
# Note: connect to T4 runtime to use GPU to speed up training times
# Start notebook, install dependencies, then runtime->restart runtime in google colab settings -> 
# Then run rest of code *without* reinstalling dependencies to avoid errors
from google.colab import files
uploaded = files.upload()

Saving korean_anki_deck.txt to korean_anki_deck (1).txt


In [ ]:
# read data from .csv file
index = 0
rows = []
with open("korean_anki_deck.txt", 'r', encoding="utf-8") as f:
    for line in f:
        en = line.split('\t')[1]
        ko = line.split('\t')[0]
        # a few examples have no English translation, so filter those out
        if en == '':
            continue
        row = {'id': index, 'translation': {'en':en, 'ko':ko}}
        rows.append(row)
        index += 1

In [ ]:
dataset_anki = DatasetDict({'train': Dataset.from_list(rows)})
dataset_anki

In [ ]:
split_datasets_anki = dataset_anki["train"].train_test_split(train_size = .9, seed = 20)
split_datasets_anki['validation'] = split_datasets_anki.pop('test')
split_datasets_anki

In [ ]:
# Choose model for korean to english translation
model_checkpoint = 'Helsinki-NLP/opus-mt-ko-en'

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, return_tensors = 'pt')


tokenizer_config.json:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/842k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/813k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.72M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [ ]:

def preprocess_function(examples):
    inputs = [example["ko"] for example in examples["translation"]]
    targets = [example["en"] for example in examples["translation"]]
    model_inputs = tokenizer(
        inputs,
        text_target=targets,
        truncation = True
    )

    return model_inputs

In [ ]:
tokenized_datasets_anki = split_datasets_anki.map(
    preprocess_function,
    batched=True,
    # defines columns to remove with following names from the dataset
    remove_columns=split_datasets_anki["train"].column_names
)

Map:   0%|          | 0/2170 [00:00<?, ? examples/s]

Map:   0%|          | 0/242 [00:00<?, ? examples/s]

In [ ]:
tokenized_datasets_anki

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 2170
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 242
    })
})

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model = model)

dict_keys(['input_ids', 'attention_mask', 'labels', 'decoder_input_ids'])

Use BLEU score as evaluation metric

In [ ]:

metric = evaluate.load('sacrebleu')

In [ ]:
def compute_metrics(eval_preds):
    preds, labels = eval_preds
    # in case the model returns more than the prediciton logits
    if isinstance(preds, tuple):
        preds = preds[0]

    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens = True)

    # replace -100s in the labels to regular pad_token_id as we can't decode -100s
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens = True)

    # remove leading/trailing whitespace
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [[label.strip()] for label in decoded_labels]

    result = metric.compute(predictions=decoded_preds, references = decoded_labels)
    return {"bleu": result["score"]}



In [ ]:
from transformers import Seq2SeqTrainingArguments

args = Seq2SeqTrainingArguments(
    f"finetuned-kde4-ko-to-en",
    eval_strategy="no",
    save_strategy="epoch",
    learning_rate = 2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,
    weight_decay = .01,
    save_total_limit = 3,
    num_train_epochs=2,
    predict_with_generate=True,
    fp16=True,
)

In [ ]:
from transformers import Seq2SeqTrainer
trainer_anki = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets_anki["train"],
    eval_dataset=tokenized_datasets_anki["validation"],
    data_collator=data_collator,
    tokenizer = tokenizer,
    compute_metrics = compute_metrics,
)

In [ ]:
max_length = 128

In [ ]:
trainer_anki.evaluate(max_length = max_length)
# Result: default starting BLEU score is 26.8 (decent translation with significant gramatical errors)
# (50-60 is considered very high quality translation, similar to human translations)

{'eval_loss': 1.0479423999786377,
 'eval_bleu': 35.58610248687253,
 'eval_runtime': 11.0983,
 'eval_samples_per_second': 21.805,
 'eval_steps_per_second': 1.442,
 'epoch': 2.0}

In [ ]:
trainer_anki.train()

Step,Training Loss
500,1.088600


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512, 'num_beams': 6, 'bad_words_ids': [[65000]], 'forced_eos_token_id': 0}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512, 'num_beams': 6, 'bad_words_ids': [[65000]], 'forced_eos_token_id': 0}


TrainOutput(global_step=544, training_loss=1.0743724808973425, metrics={'train_runtime': 79.2763, 'train_samples_per_second': 54.745, 'train_steps_per_second': 6.862, 'total_flos': 16185409929216.0, 'train_loss': 1.0743724808973425, 'epoch': 2.0})

In [ ]:
trainer_anki.evaluate(max_length = max_length)
# Result: Final BLEU score after fine-tuning is 35.6 (considered to be a good translation)

{'eval_loss': 1.6353167295455933,
 'eval_bleu': 47.749822262505624,
 'eval_runtime': 50.0356,
 'eval_samples_per_second': 0.799,
 'eval_steps_per_second': 0.06,
 'epoch': 2.0}

In [ ]:
# saving fine-tuned model
trainer_anki.save_model("gdrive/My Drive/finetuned-kde4-en-to-ko2")

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512, 'num_beams': 6, 'bad_words_ids': [[65000]], 'forced_eos_token_id': 0}


In [ ]:
fine_tuned_model = AutoModelForSeq2SeqLM.from_pretrained("gdrive/My Drive/finetuned-kde4-en-to-ko2", local_files_only=True)

Use new model for translations:

In [ ]:
from transformers import pipeline
p = pipeline('translation', model = "gdrive/My Drive/finetuned-kde4-en-to-ko2")

In [ ]:
p = pipeline('translation', model = model_checkpoint)

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [ ]:
p("무거워서 떨어뜨릴 뻔 했어요")


[{'translation_text': "I was so heavy I'd have to drop it."}]